In [1]:
#import library
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torchvision import models

In [2]:
#Load the DataLoader objects
with open(r"C:\Users\user\Documents\!TA\!TA\all trial\train_loader.pkl", "rb") as f:
    train_loader = pickle.load(f)
with open(r"C:\Users\user\Documents\!TA\!TA\all trial\valid_loader.pkl", "rb") as f:
    valid_loader = pickle.load(f)

In [3]:
#Confirm the DataLoaders are loaded
print(f"Train Loader: {len(train_loader)} batches")
print(f"Valid Loader: {len(valid_loader)} batches")

Train Loader: 37 batches
Valid Loader: 10 batches


In [4]:
#Check device availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
# #load the pre-trained VGG16 model architecture
# vgg16 = models.vgg16(weights=None).to(device)

# #Modify the classifier layer for 3 output classes
# vgg16.classifier[6] = nn.Linear(in_features=4096, out_features=3).to(device)

# #Load the saved state dictionary
# vgg16.load_state_dict(torch.load("vgg16_state_dict.pth", map_location=device))

#load the pre-trained VGG16 model architecture
vgg11 = models.vgg11(weights=None).to(device)

#Modify the classifier layer for 3 output classes
vgg11.classifier = nn.Sequential(
    nn.Linear(in_features=512 * 7 * 7, out_features=4096),
    nn.ReLU(),
    nn.Dropout(p=0.4),
    nn.Linear(in_features=4096, out_features=4096),
    nn.ReLU(),
    nn.Dropout(p=0.4),
    nn.Linear(in_features=4096, out_features=3)
).to(device)

 #Load the saved state dictionary if available
try:
    vgg11.load_state_dict(torch.load("vgg11_with_dropout_state_dict.pth", map_location=device))
    print("Pretrained weights loaded successfully.")
except FileNotFoundError:
    print("No pretrained weights found. Training from scratch.")

C:\Users\user\AppData\Local\Temp\ipykernel_17508\3150811408.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg11.load_state_dict(torch.load("vgg11_with_dropout_state_d

Pretrained weights loaded successfully.


In [6]:
#Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
# optimizer = optim.Adam(vgg16.classifier.parameters(), lr=0.001)
# optimizer = optim.Adam(vgg11.classifier.parameters(), lr=0.001)
optimizer = optim.Adam([
   {'params': vgg11.features[11:].parameters(), 'lr': 1e-6},  # last conv block
   {'params': vgg11.classifier.parameters(),    'lr': 1e-3}
], weight_decay=1e-5)

In [7]:
#Learning rate scheduler
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Reduce lr by 0.5 every 5 epochs

# Adaptive learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5, verbose=True)


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [8]:
# Training function with early stopping
def train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs=10, patience=5):
    best_accuracy = 0.0
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_train = 0

        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            total_train += labels.size(0)

        train_accuracy = 100 * train_correct / total_train

        # Validation loop
        model.eval()
        valid_correct = 0
        total_valid = 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                valid_correct += (preds == labels).sum().item()
                total_valid += labels.size(0)

        valid_accuracy = 100 * valid_correct / total_valid

        # Print stats for the epoch
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {train_loss / len(train_loader):.4f}, "
              f"Train Accuracy: {train_accuracy:.2f}%, "
              f"Validation Accuracy: {valid_accuracy:.2f}%")

        # Early stopping logic
        if valid_accuracy > best_accuracy:
            best_accuracy = valid_accuracy
            torch.save(model.state_dict(), "best_vgg11_with_dropout_model.pth")
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}!")
            break

        # Update learning rate scheduler
        scheduler.step(valid_accuracy)

    print(f"Best Validation Accuracy: {best_accuracy:.2f}%")

In [ ]:
#Train the model
#train_model(vgg16, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs=10)
train_model(vgg11, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs=10, patience=5)